## Compulsory Task 

In this compulsory task you will clean the country column and parse the date column in the **store_income_data_task.csv** file.

In [50]:
# Load up store_income_data.csv

import pandas as pd
import numpy as np

import fuzzywuzzy
from fuzzywuzzy import process

import chardet

df = pd.read_csv('store_income_data_task.csv', index_col = 0) 


1. Take a look at all the unique values in the "country" column. Then, convert the column to lowercase and remove any trailing white spaces.

In [51]:
# Looking at the number of unique values in the country column.
unique_countries_count = df["country"].nunique()
print(f"There are {unique_countries_count} unique countries")

# Converting the column into lowercase.
df["country"] = df["country"].str.lower()

# Removing any whitespace.
df["country"] = df["country"].str.strip()



There are 76 unique countries


2. Note that there should only be three separate countries. Eliminate all variations, so that 'South Africa', 'United Kingdom' and 'United States' are the only three countries.

In [52]:
# Set seed for reproducibility
np.random.seed(0)

countries = df['country'].unique()
print(f"There are {len(countries)} unique countries")
countries



There are 37 unique countries


array(['united states/', 'britain', 'united states', 'britain/',
       'united kingdom', 'u.k.', 'sa', 'u.k/', 'america', nan, 's.a.',
       'england', 'uk', 's.a./', 'u.k', 'america/', 'sa.', '', 'uk.',
       'england/', 'united states of america', 'uk/', 'sa/', 'england.',
       'america.', 's.a..', 'united states of america.',
       'united states of america/', 'united states.',
       's. africasouth africa', 'britain.', '/', 'united kingdom.',
       's. africasouth africa/', 'united kingdom/',
       's. africasouth africa.', '.'], dtype=object)

In [53]:
# Convert to lower case
df['country'] = df['country'].str.lower()

# Remove trailing white spaces
df['country'] = df['country'].str.strip()

# Get the top 10 closest matches to "united kingdom"
matches = fuzzywuzzy.process.extract("uk", countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

# Inspect matches
matches
df.head()

,store_name,store_email,department,income,date_measured,country
id,,,,,,
1,"Cullen/Frost Bankers, Inc.",NaN,Clothing,$54438554.24,4-2-2006,united states
2,Nordson Corporation,NaN,Tools,$41744177.01,4-1-2006,britain
3,"Stag Industrial, Inc.",NaN,Beauty,$36152340.34,12-9-2003,united states
4,FIRST REPUBLIC BANK,ecanadine3@fc2.com,Automotive,$8928350.04,8-5-2006,britain
5,Mercantile Bank Corporation,NaN,Baby,$33552742.32,21-1-1973,united kingdom


In [54]:
# Function to replace rows in the provided column of the provided DataFrame
# that match the provided string above the provided ratio with the provided string
def replace_matches_in_column(df, column, string_to_match, min_ratio = 90):
    # get a list of unique strings
    strings = df[column].unique()
    
    # Get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # Only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # Get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # Replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match
    
    # Let us know when the function is done
    print("All done!")

replace_matches_in_column(df=df, column='country', string_to_match="united kingdom")
replace_matches_in_column(df=df, column='country', string_to_match="united states")
replace_matches_in_column(df=df, column='country', string_to_match="united states of america")
replace_matches_in_column(df=df, column='country', string_to_match="south africa")
replace_matches_in_column(df=df, column='country', string_to_match="uk")

All done!
All done!
All done!
All done!
All done!


In [62]:
# get all the unique values in the 'country' column
countries = df['country'].unique()

# Replace . or / with white spaces
df['country'] = df['country'].str.replace(".", " ").str.replace("/", " ")

print(f"There are {len(countries)} unique countries")
countries

There are 18 unique countries


array(['united states', 'britain', 'britain ', 'united kingdom', 'u k ',
       'sa', nan, 's a ', 'england', 's a  ', 'america ', 'sa ', '',
       'england ', 'united states of america', 's  africasouth africa',
       ' ', 's  africasouth africa '], dtype=object)

In [80]:
df.replace('uk', 'united kingdom', inplace=True)
df.replace("u k" ,'united kingdom', inplace=True)
df.replace("britain" ,'united kingdom', inplace=True)
df.replace("britain " ,'united kingdom', inplace=True)
df.replace("u k " ,'united kingdom', inplace=True)
df.replace("england" ,'united kingdom', inplace=True)
df.replace("england " ,'united kingdom', inplace=True)
df.replace('united states of america', 'united states', inplace=True)
df.replace('united states', 'united states', inplace=True)
df.replace('america', 'united states', inplace=True)
df.replace('america ', 'united states', inplace=True)
df.replace('sa', 'south africa', inplace=True)
df.replace('s  africasouth africa', 'south africa', inplace=True)
df.replace('s a', 'south africa', inplace=True)
df.replace('s a ', 'south africa', inplace=True)
df.replace('s a  ', 'south africa', inplace=True)
df.replace('sa ', 'south africa', inplace=True)

# Define the list of countries to keep
valid_countries = ['united states', 'united kingdom', 'south africa']

# Filter the DataFrame to keep only rows with valid countries
df_filtered = df[df['country'].isin(valid_countries)]

# Replace any other values with NaN
df_filtered['country'] = df_filtered['country'].replace('', np.nan)
df_filtered['country'] = df_filtered['country'].replace(' ', np.nan)
df_filtered['country'] = df_filtered['country'].replace('s  africasouth africa ', np.nan)

# Drop rows with NaN values in the 'country' column
df_filtered = df_filtered.dropna(subset=['country'])

# Check the result
print(df_filtered['country'].unique())

df.head()


['united states' 'united kingdom' 'south africa']


C:\Users\A-C19\AppData\Local\Temp\ipykernel_2288\1177479497.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['country'] = df_filtered['country'].replace('', np.nan)
C:\Users\A-C19\AppData\Local\Temp\ipykernel_2288\1177479497.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['country'] = df_filtered['country'].replace(' ', np.nan)
C:\Users\A-C19\AppData\Local\Temp\ipykernel_2288\1177479497.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

,store_name,store_email,department,income,date_measured,country
id,,,,,,
1,"Cullen/Frost Bankers, Inc.",NaN,Clothing,54438554.24,4-2-2006,united states
2,Nordson Corporation,NaN,Tools,41744177.01,4-1-2006,united kingdom
3,"Stag Industrial, Inc.",NaN,Beauty,36152340.34,12-9-2003,united states
4,FIRST REPUBLIC BANK,ecanadine3@fc2.com,Automotive,8928350.04,8-5-2006,united kingdom
5,Mercantile Bank Corporation,NaN,Baby,33552742.32,21-1-1973,united kingdom


3. Create a new column called `days_ago` in the DataFrame that is a copy of the 'date_measured' column but instead it is a number that shows how many days ago it was measured from the current date. Note that the current date can be obtained using `datetime.date.today()`.

In [99]:
import pandas as pd
from datetime import date

# Create a copy of the DataFrame to avoid modifying the original data
df_copy = df.copy()

# Convert the 'date_measured' column to datetime format
df_copy['date_measured'] = pd.to_datetime(df['date_measured'], errors='coerce')

# Calculate the difference in days for each date from the current date
current_date = date.today()
df_copy['days_ago'] = (current_date - (df_copy['date_measured'].dt.date))

# Print the DataFrame to see the new column
print(df_copy.head())





                     store_name         store_email  department       income  \
id                                                                             
1    Cullen/Frost Bankers, Inc.                 NaN    Clothing  54438554.24   
2           Nordson Corporation                 NaN       Tools  41744177.01   
3         Stag Industrial, Inc.                 NaN      Beauty  36152340.34   
4           FIRST REPUBLIC BANK  ecanadine3@fc2.com  Automotive   8928350.04   
5   Mercantile Bank Corporation                 NaN        Baby  33552742.32   

   date_measured         country            days_ago  
id                                                    
1     2006-04-02   united states  6554 days, 0:00:00  
2     2006-04-01  united kingdom  6555 days, 0:00:00  
3     2003-12-09   united states  7399 days, 0:00:00  
4     2006-08-05  united kingdom  6429 days, 0:00:00  
5            NaT  united kingdom                 NaN  
